In [10]:
!pip install selenium beautifulsoup4 markdownify

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
textract 1.6.5 requires beautifulsoup4~=4.8.0, but you have beautifulsoup4 4.12.2 which is incompatible.
textract 1.6.5 requires six~=1.12.0, but you have six 1.16.0 which is incompatible.



     ------------------------------------ 143.0/143.0 kB 340.4 kB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.8.2
    Uninstalling beautifulsoup4-4.8.2:
      Successfully uninstalled beautifulsoup4-4.8.2


In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import json
import markdownify

class Web_Scraping():
    def scraping(self):
        # Create lists to store the data
        data_list = []

        # Configure Chrome options
        options = Options()
        options.add_argument("--headless")  # Run Chrome in headless mode (no GUI)
        options.add_argument("--start-maximized")  # Maximize the browser window

        # Initialize the web driver
        driver = webdriver.Chrome(options=options)

        # Loop until no more pages are found
        i = 1
        while True:
            url = f"https://www.barelaw.in/indian-courts-2/?p={i}"
            driver.get(url)

            # Wait for page to load (you may need to adjust the waiting time)
            driver.implicitly_wait(10)  # Wait for up to 10 seconds for elements to load

            # Get page source after waiting
            page_source = driver.page_source

            # Use BeautifulSoup to parse the page source
            soup = BeautifulSoup(page_source, "html.parser")

            # Check if there is no content on the current page, indicating the end of pagination
            if not soup.select('.size-normal'):
                break

            # Extract titles and content
            title_elements = soup.select('.size-normal')
            content_elements = soup.select('#wi-content p')

            # Convert HTML tables to Markdown
            table_elements = soup.select('#wi-content table')
            for table in table_elements:
                markdown_table = markdownify.markdownify(str(table))
                content_elements.append(markdown_table)

            # Combine titles and content into a dictionary
            page_data = {
                "title": [title.text.strip() for title in title_elements],
                "content": [content.text.strip() for content in content_elements]
            }

            # Append the data to the list
            data_list.append(page_data)

            i += 1

        # Close the web driver
        driver.quit()

        # Save the scraped data to a JSON file
        with open("scraped_data.json", "w") as json_file:
            json.dump(data_list, json_file, indent=4)
            
        print("Data scraped and saved to 'scraped_data.json'")
        return "scrapping successful"
    
if __name__ == "__main__":
    process = Web_Scraping()
    process.scraping()

Data scraped and saved to 'scraped_data.json'
